In [31]:
import requests
import json

# ✅ 발급받은 인증키 (예: 일반 인증키)
API_KEY = "534a64795371717134386649476a45"
station_name = "강남"  # 원하는 역 이름

# ✅ URL 구성
url = f"https://swopenapi.seoul.go.kr/api/subway/{API_KEY}/json/realtimeStationArrival/0/5/{station_name}"

# ✅ 요청 및 응답
try:
    response = requests.get(url, timeout=5)
    response.raise_for_status()
    data = response.json()

    if "realtimeArrivalList" in data:
        print(f"📍 {station_name}역 실시간 도착 정보:")
        for train in data["realtimeArrivalList"]:
            print(f"- {train['trainLineNm']} | {train['arvlMsg2']} | {train['arvlCd']}")
    else:
        print("🚫 도착 정보가 없습니다.")

except requests.exceptions.Timeout:
    print("⏱ 요청 시간이 초과되었습니다.")
except requests.exceptions.RequestException as e:
    print(f"❌ 요청 오류: {e}")


⏱ 요청 시간이 초과되었습니다.


In [25]:
import requests
import csv
from collections import defaultdict, deque

# 🔹 CSV 기반 그래프 생성
def build_graph_from_csv(filepath):
    graph = defaultdict(list)
    with open(filepath, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            from_station = row['출발역']
            to_station = row['도착역']
            graph[from_station].append(to_station)
            graph[to_station].append(from_station)  # 왕복 가능
    return graph

# 🔹 BFS로 정거장 수 계산
def get_station_count(graph, start, end):
    queue = deque([(start, 0)])
    visited = set([start])

    while queue:
        station, depth = queue.popleft()
        if station == end:
            return depth
        for neighbor in graph[station]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, depth + 1))
    return -1  # 경로 없음

# 🔹 실시간 도착 정보 불러오기
def get_real_time_info(station_name, api_key):
    url = f"https://swopenapi.seoul.go.kr/api/subway/{api_key}/json/realtimeStationArrival/0/5/{station_name}"
    res = requests.get(url)
    if res.status_code != 200:
        return None
    data = res.json()
    return data.get("realtimeArrivalList", [])

# 🔹 통합 실행 예시
def main():
    start_station = "강남"
    end_station = "삼성"
    api_key = "534a64795371717134386649476a45"
    csv_path = "station_list_utf8.csv"

    # 실시간 열차 도착 정보
    arrival_list = get_real_time_info(start_station, api_key)
    print(f"🚇 [{start_station}] 도착 정보:")
    for item in arrival_list:
        print(f"- {item['trainLineNm']} / {item['arvlMsg2']}")

    # 정거장 수 계산
    graph = build_graph_from_csv(csv_path)
    stop_count = get_station_count(graph, start_station, end_station)
    print(f"\n📍 출발역: {start_station} → 도착역: {end_station}")
    print(f"🛤️ 총 정거장 수: {stop_count}개")

if __name__ == "__main__":
    main()


ConnectTimeout: HTTPSConnectionPool(host='swopenapi.seoul.go.kr', port=443): Max retries exceeded with url: /api/subway/534a64795371717134386649476a45/json/realtimeStationArrival/0/5/%EA%B0%95%EB%82%A8 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000024F7FECAF90>, 'Connection to swopenapi.seoul.go.kr timed out. (connect timeout=None)'))